In [1]:
import numpy as np
import os
from PIL import Image
from torch.utils.data import Dataset
import skimage.io as io
import glob
import torch
import torchvision
from torchvision.models import vgg16,resnet101
from torchvision.transforms import Compose,CenterCrop,Normalize,Scale
from torchvision.transforms import ToTensor,ToPILImage,Resize
from torch.utils.data import DataLoader
from torch.optim import SGD,Adam
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from torch.nn import init
from tqdm import tqdm_notebook,tqdm

In [2]:
def load_image(file):
    return Image.open(file)

class yugioh(Dataset):
    def __init__(self,data_dir,data_txt,label_txt,input_transform=None):
        self.data_dir=data_dir
        self.input_transform=input_transform
        self.image_names=np.genfromtxt(data_txt,dtype=int)
        self.label_txt=label_txt
        self.label=np.genfromtxt(label_txt,dtype=int)
    def __getitem__(self,index):
        filename=self.image_names[index]
        image=load_image(self.data_dir+str(filename)+".jpg").convert('RGB')
        if self.input_transform is not None:
            image=self.input_transform(image)
        return image,self.label[index]

    def __len__(self):
        return len(self.image_names)


In [3]:
class resnet(nn.Module):
    def __init__(self):
        super(resnet,self).__init__()
        self.pretrained_model=resnet101(pretrained=True)
        self.pretrained_model.fc= nn.Linear(in_features=2048, out_features=1)
    def forward(self,x):
        res=self.pretrained_model(x)
        return F.sigmoid(res)
 

model=resnet().cuda()

#print(model)
model=torch.nn.DataParallel(model,device_ids=[0,1])


data_dir="dataset/"
train_data_path="train.txt"
train_label_path="train_label.txt"
test_data_path="test.txt"
test_label_path="test_label.txt"
val_data_path="val.txt"
val_label_path="val_label.txt"

input_transform=Compose([
                        Resize((224,224)),
                        ToTensor(),
                        Normalize([.485,.456,.406],[.229,.224,.225])
])

epoches=100
batch_size=64


train_loader=DataLoader(yugioh(data_dir,train_data_path,train_label_path,input_transform),
                      num_workers=4,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(yugioh(data_dir,val_data_path,val_label_path,input_transform),
                      num_workers=4,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(yugioh(data_dir,test_data_path,test_label_path,input_transform),
                      num_workers=4,batch_size=batch_size,shuffle=True)

criterion=nn.L1Loss()
optimizer=Adam(model.parameters(),1e-4)

def validate(model,batch_size):
    model.eval()
    losses=[]
    for step,(images,labels) in enumerate(tqdm(val_loader)):
        labels=labels.float()/12
        labels=labels.cuda()
        labels=labels.view(-1,1)
        images=images.cuda()
        outputs=model(images)
        loss=criterion(outputs,labels)
        losses.append(loss.data.cpu().numpy())
    print("eval loss: ",np.mean(np.array(losses)))

def test(model,batch_size):
    model.eval()
    losses=[]
    for step,(images,labels) in enumerate(tqdm(test_loader)):
        labels=labels.float()/12
        labels=labels.cuda()
        labels=labels.view(-1,1)
        images=images.cuda()
        outputs=model(images)
        loss=criterion(outputs,labels)
        losses.append(loss.data.cpu().numpy())
    print("--------------------------------------")
    print("test loss: ",np.mean(np.array(losses)))
    print("--------------------------------------")
    
    
def train(model,batch_size,epoches):
    for epoch in range(1,epoches+1):
        model.train()
        losses=[]
        print("Epoch: ",epoch)
        for step,(images,labels) in enumerate(tqdm(train_loader)):
            labels=labels.float()/12
            labels=labels.cuda()
            labels=labels.view(-1,1)
            images=images.cuda()
            optimizer.zero_grad()
            outputs=model(images)
            loss=criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            losses.append(loss.data.cpu().numpy())
        print("train loss: ",np.mean(np.array(losses)))
        validate(model,batch_size)
        torch.save(model.state_dict(),"model/epoch{}.pth".format(epoch))
    test(model,batch_size)
        

In [ ]:
train(model,batch_size,epoches)

  0%|          | 0/125 [00:00<?, ?it/s]

Epoch:  1


  0%|          | 0/19 [00:00<?, ?it/s]

train loss:  0.1320405


100%|██████████| 19/19 [00:04<00:00,  4.18it/s]


...

  0%|          | 0/19 [00:00<?, ?it/s]

train loss:  0.0065133087


100%|██████████| 19/19 [00:04<00:00,  4.19it/s]


eval loss:  0.06402217
Epoch:  99


 90%|████████▉ | 112/125 [01:09<00:08,  1.61it/s]